# Clustering

## Dependencias

In [ ]:
!pip install kneed numpy matplotlib scikit-learn seaborn pandas folium

In [ ]:
import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns 
import warnings
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from kneed import KneeLocator

In [ ]:
warnings.filterwarnings("ignore")

## Segmentación de clientes con K-Means

Usaremos o algoritmo K-Means para segmentar un datasets con datos de compras.

### Datos

#### Exploración dos datos

Descargamos o dataset co que traballaremos.

In [ ]:
!wget http://fegalaz.usc.es/~sdocio/apau2/p1/datasets/customers.csv

In [ ]:
df = pd.read_csv(r'customers.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.drop(columns=["CustomerID"]).describe()

In [ ]:
df.dtypes

Comprobamos se hai valores nulos no dataset.

In [ ]:
df.isnull().sum()

#### Visualización de datos

In [ ]:
plt.figure(figsize = (12, 3))
cols = ['Idade' , 'Ingresos anuais (USD)' , 'Perfil de gasto (1-100)']
for i, x in enumerate(['Age', 'Annual Income (k$)', 'Spending Score (1-100)']):
    plt.subplot(1, 3, i+1)
    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    sns.histplot(df[x], bins=20, kde=True)
    plt.title('Gráfica de {}'.format(cols[i]))
plt.show()

### Segmentación usando a idade e o perfil de gasto

Normalizamos os datos para axustar a escala.

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(df[['Age' , 'Spending Score (1-100)']])

**Orixinal**

In [ ]:
df[['Age' , 'Spending Score (1-100)']][:10].values

**Normalizado**

In [ ]:
X[:10]

In [ ]:
X.shape

#### Método do cóbado

In [ ]:
sse = []

for k in range(2, 20):
  km2 = KMeans(n_clusters=k, random_state=1234)
  km2.fit(X)
  sse.append(km2.inertia_)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(range(2, 20), sse, marker='o')
plt.title('Método do cóbado')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inertia ou SSE')
plt.xticks(range(2, 20, 2))
plt.show()

In [ ]:
kl = KneeLocator(range(2, 20), sse, curve="convex", direction="decreasing")
kl.elbow

#### Silhouette Score

In [ ]:
silhouette_coefficients = []

for k in range(2, 20):
     km3 = KMeans(n_clusters=k)
     km3.fit(X)
     score = silhouette_score(X, km3.labels_)
     silhouette_coefficients.append(score)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(range(2, 20), silhouette_coefficients, marker='o')
plt.title('Silhouette Score')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Silhouette')
plt.xticks(range(2, 20, 2))
plt.show()

#### Clustering

In [ ]:
km = KMeans(n_clusters=kl.elbow, random_state=1234)
km.fit(X)
labels = km.labels_

In [ ]:
plt.figure(figsize=(15, 7))
plt.scatter(x='Age', y='Spending Score (1-100)', data=df, c=labels, s=100)
plt.ylabel('Perfil de gasto (1-100)') , plt.xlabel('Idade')
plt.show()

### Segmentación usando perfil de gasto e ingresos anuais

In [ ]:
scaler = MinMaxScaler()
Z = scaler.fit_transform(df[['Annual Income (k$)' , 'Spending Score (1-100)']])

In [ ]:
sse = []

for n in range(2, 20):
    km3 = KMeans(n_clusters=n, init='k-means++', random_state=1234)
    km3.fit(Z)
    sse.append(km3.inertia_)

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(range(2, 20), sse, marker='o')
plt.title('Método do cóbado')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inertia ou SSE')
plt.xticks(range(2, 20, 2))
plt.show()

In [ ]:
kl = KneeLocator(range(2, 20), sse, curve="convex", direction="decreasing")
kl.elbow

In [ ]:
km4 = KMeans(n_clusters=kl.elbow, init='k-means++', random_state=1234)
km4.fit(Z)
labels2 = km4.labels_

In [ ]:
plt.figure(figsize=(12, 6) )
plt.clf()
plt.scatter(x='Annual Income (k$)', y='Spending Score (1-100)', data=df, c=labels2, s=200)
plt.ylabel('Perfil de gasto (1-100)') , plt.xlabel('Ingresos anuais (k$)')
plt.show()

## Clustering de datos xeolocalizados

Temos un dataset de clientes dun servizo de transporte similar a Uber na cidade de Fargo, que inclúe información sobre lugar de residencia e traballo. Queremos segmentar a poboación de estudantes polo seu lugar de residencia.

### Datos

#### Exploración dos datos

In [ ]:
!wget http://fegalaz.usc.es/~sdocio/apau2/p1/datasets/riders_data.csv

In [ ]:
df2 = pd.read_csv(r'riders_data.csv')

In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
df2.dtypes

Comprobamos se hai valores nulos no dataset.

In [ ]:
df2.isnull().sum()

Seleccionamos os clientes que son estudantes.

In [ ]:
students = df2[df2['student'] == 1]

In [ ]:
students.head()

In [ ]:
students.shape

Comprobamos que as entradas correspondentes a estudantes carecen de coordenadas para o lugar de traballo, mais non hai nulos no de residencia, que é co que traballaremos.

In [ ]:
students.isnull().sum()

Extraemos as *features* que nos interesan.

In [ ]:
X = students[['home_lat', 'home_lon']].values

In [ ]:
X[:5]

#### Visualización de datos

Neste caso o método que escollemos para determinar o número óptimo de *clusters* é o de visualizar os datos. Como traballamos con coordenadas, produciremos un mapa no que engadiremos os puntos de datos.

In [ ]:
avg_lat = students['home_lat'].mean()
avg_lon = students['home_lon'].mean()
cluster_map = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)

In [ ]:
for idx, row in students.iterrows():
    folium.CircleMarker(
        location=[row['home_lat'], row['home_lon']],
        radius=5,
        color='blue',
        fill=True,
        tooltip=f"Student: {row['first_name']} {row['last_name']}"
    ).add_to(cluster_map)

In [ ]:
cluster_map

### Clustering por lugar de residencia

Para facer o *clustering* usaremos GaussianMixture (GMM), un algoritmo de *clustering* probabilístico que modela os datos como unha combinación de varias distribucións normais (gaussianas). A diferenza de K-Means, GMM permite que un punto pertenza a varios clusters con diferentes probabilidades.

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(X)

In [ ]:
np.set_printoptions(suppress=True)
results = gmm.predict_proba(X)
results[:5]

Engadimos as predicións ao Dataframe.

In [ ]:
students['cluster'] = gmm.predict(X)
students['probability'] = gmm.predict_proba(X).max(axis=1)

In [ ]:
students[:3]

Agora que temos os puntos do dataset asignado a un *cluster* en función da probabilidade obtida, podemos volver producir o mesmo mapa, agora mudando a cor de cada punto de datos en función do *cluster* ao que foi asignado.

Comezamos colocando os centroides.

In [ ]:
for center in gmm.means_:
    folium.Marker(
        location=center,
        icon=folium.Icon(color='lightblue', icon='cloud'),
        tooltip=f'Cluster Center: {center}'
    ).add_to(cluster_map)

In [ ]:
color_map = {0: 'green', 1: 'blue', 2: 'red', 3: 'yellow', 4: 'darkred', 5: 'lightred', 6: 'beige', 7: 'darkblue'}
for idx, row in students.iterrows():
    folium.CircleMarker(
        location=[row['home_lat'], row['home_lon']],
        radius=5,
        color=color_map[row['cluster']],
        fill=True,
        tooltip=f"""
        Student: {row['first_name']} {row['last_name']}
        Cluster: {row['cluster']}
        Probability: {row['probability']:.2f}
        """
    ).add_to(cluster_map)

In [ ]:
cluster_map